## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [1]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [2]:
# load data
large = True

if large:
    path = "/large"
else:
    path = ""

players = pd.read_csv("../data/optim-data" + path + "/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
market_values = pd.read_csv("../data/optim-data" + path + "/0_selection_mv.csv", index_col=0,).drop(['player_id', 'trial'], axis=1).values
# salaries = pd.read_csv("../data/optim-data/0_selection_salary.csv", index_col=0).drop('Player', axis=1).values
n_time = market_values.shape[1]
n_trials = int(market_values.shape[0]/n_players)
market_values = market_values.reshape((n_trials, n_players, n_time))

salaries = np.median(market_values, axis=0)*0.1

# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [3]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 37
starting_budget = 995000000/1.141
wage_rev = 0.63
rho = 0.05
ret = 0.02

time = np.arange(n_time)
trials = np.arange(n_trials)

In [5]:
# Decision variables
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")
lent_out = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "lentout_%s_%s")
borrowed = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "borrowed_%s_%s")

budget = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Budget_%s_%s", lb=0)
team_value = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Team Value_%s_%s")

sunset_value = ftcp.continuous_var_list(keys=trials, name = "Sunset Value_%s")
optim_value = ftcp.continuous_var(name = "Optimisation Value")

In [6]:
# Non decision variables
# Total number of players owned by the team is not the same as number of players playing in tournament
N_owned = ftcp.integer_var_list(keys = time, name = 'Number of Players owned by the team_%s', ub=30)
N_team = ftcp.integer_var_list(keys = time, name = 'Number of Players in the team_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [7]:
# Set up Constraints for number of players
for year in time:
    for trial in trials:
        ###
        # Not used in optim, but will be used as constraint to control costs
        ###
        previous_budget = starting_budget if year == 0 else budget[year-1, trial]
        ftcp.add_constraint(budget[year, trial] == previous_budget * (1 + ret)
                            + (1/wage_rev - 1) * ftcp.sum((assigned[index, year] + 1.1*borrowed[index, year] - 0.1*lent_out[index, year]) * salaries[index, year] for index in range(n_players)))
        
        ftcp.add_constraint(team_value[year, trial] == ftcp.sum((assigned[index, year] + borrowed[index, year]) * market_values[trial, index, year] for index in range(n_players))/((1 + rho)**year)
                            - ftcp.sum((assigned[index, year] + 1.1*borrowed[index, year] - 0.1*lent_out[index, year]) * salaries[index, year] for index in range(n_players)))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['FW'] for index, player in players.iterrows()))
    
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_team[year])
    ftcp.add_constraint(N_owned[year] == ftcp.sum(assigned[index, year] for index, player in players.iterrows()))
    
    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age)
        
        ftcp.add_constraint(assigned[index, year] <= (player['Raritan']))
        ftcp.add_constraint(lent_out[index, year] <= assigned[index, year])
        ftcp.add_constraint(borrowed[index, year] <= (1 - player['Raritan']))
        
for trial in trials:        
    ftcp.add_constraint(sunset_value[trial] == ftcp.sum((assigned[index, time[-1]] + borrowed[index, time[-1]]) * market_values[trial, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
    
ftcp.add_constraint(optim_value == (ftcp.sum(team_value[x, y] for x in time for y in trials) + ftcp.sum(sunset_value[x] for x in trials))/n_trials)

docplex.mp.LinearConstraint[](Optimisation Value,EQ,0.001Team Value_0_0+0.001Team Value_0_1+0.001Team Value_0_2+0.001Team Value_0_3+0.001Team Value_0_4+0.001Team Value_0_5+0.001Team Value_0_6+0.001Team Value_0_7+0.001Team Value_0_8+0.001Team Value_0_9+0.001Team Value_0_10+0.001Team Value_0_11+0.001Team Value_0_12+0.001Team Value_0_13+0.001Team Value_0_14+0.001Team Value_0_15+0.001Team Value_0_16+0.001Team Value_0_17+0.001Team Value_0_18+0.001Team Value_0_19+0.001Team Value_0_20+0.001Team Value_0_21+0.001Team Value_0_22+0.001Team Value_0_23+0.001Team Value_0_24+0.001Team Value_0_25+0.001Team Value_0_26+0.001Team Value_0_27+0.001Team Value_0_28+0.001Team Value_0_29+0.001Team Value_0_30+0.001Team Value_0_31+0.001Team Value_0_32+0.001Team Value_0_33+0.001Team Value_0_34+0.001Team Value_0_35+0.001Team Value_0_36+0.001Team Value_0_37+0.001Team Value_0_38+0.001Team Value_0_39+0.001Team Value_0_40+0.001Team Value_0_41+0.001Team Value_0_42+0.001Team Value_0_43+0.001Team Value_0_44+0.001Team Val

In [9]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 123571
   - binary=81510, integer=60, continuous=42001
 - number of constraints: 150741
   - linear=150741
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [10]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Presolve has eliminated 104875 rows and 55039 columns...
Presolve has improved bounds 12001 times...
Presolve has eliminated 129199 rows and 75078 columns...
Presolve has improved bounds 24001 times...
Tried aggregator 1 time.
Presolve has eliminated 129199 rows and 75078 columns...
Presolve has improved bounds 24001 times...
MIP Presolve eliminated 129199 rows and 75078 columns.
Reduced MIP has 21542 rows, 48493 columns, and 56839108 nonzeros.
Reduced MIP has 28422 binaries, 80 generals, 0 SOSs, and 0 indicators.
Presolve time = 73.38 sec. (38010.13 ticks)
Probing time = 0.55 sec. (474.51 ticks)
Presolve has eliminated 0 rows and 0 columns...
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 21542 rows, 48493 columns, and 56839108 nonzeros.
Reduced MIP has 28422 binaries, 80 generals, 0 SOSs, and 0 indicators.
Presolve time = 106.20 sec. (23399.09 ticks)
Probing time

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [11]:
ftcp.print_solution()


objective: 22345809969.374
  "assign_15_9"=1
  "assign_68_0"=1
  "assign_68_1"=1
  "assign_68_2"=1
  "assign_68_3"=1
  "assign_68_4"=1
  "assign_81_4"=1
  "assign_81_5"=1
  "assign_81_6"=1
  "assign_81_7"=1
  "assign_81_8"=1
  "assign_81_9"=1
  "assign_89_6"=1
  "assign_89_7"=1
  "assign_89_8"=1
  "assign_89_9"=1
  "assign_118_0"=1
  "assign_118_1"=1
  "assign_118_2"=1
  "assign_118_3"=1
  "assign_118_4"=1
  "assign_118_5"=1
  "assign_118_6"=1
  "assign_118_7"=1
  "assign_118_8"=1
  "assign_118_9"=1
  "assign_143_0"=1
  "assign_143_1"=1
  "assign_143_2"=1
  "assign_143_3"=1
  "assign_143_4"=1
  "assign_143_5"=1
  "assign_143_6"=1
  "assign_143_7"=1
  "assign_143_8"=1
  "assign_143_9"=1
  "assign_190_0"=1
  "assign_190_1"=1
  "assign_190_2"=1
  "assign_190_3"=1
  "assign_190_4"=1
  "assign_190_5"=1
  "assign_190_6"=1
  "assign_190_7"=1
  "assign_190_8"=1
  "assign_190_9"=1
  "assign_260_2"=1
  "assign_260_3"=1
  "assign_260_4"=1
  "assign_260_5"=1
  "assign_260_6"=1
  "assign_260_7"=1
 

In [12]:
ftcp.solution.export("../data/optim-data" + path + "/optim_solution.json")

In [22]:
# import docplex.mp.conflict_refiner as cr

# ftcp.solve()
# solve_status = ftcp.get_solve_status()
# if solve_status.name in ['INFEASIBLE_SOLUTION', 'INFEASIBLE_OR_UNBOUNDED_SOLUTION']:
#     cref = cr.ConflictRefiner()
#     print('show some of the constraints that can be removed to arrive at a minimal conflict')
#     cref_result = cref.refine_conflict(ftcp, display=True)  # display flag is to show the conflicts

#     cref_result.display()
#     cref_result.as_output_table()